Azure ML & Azure Databricks notebooks by René Bremer (original taken from Parashar Shah)

Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

##### In this notebook the following steps will be excuted:

1. Log metrics of models that was trained on 2000 pictures and all 60000 pictures
2. Register best model (trained with 60000 pictures)

Make sure you added libraries to azureml-sdk[databricks], Keras and TensorFlow to your cluster.

#0. Set parameters

In [0]:
workspace="newws"
resource_grp="newres"
subscription_id="Workspace-e18a6833-eeb6-4db0-858d-b098746f4034-deployment"


path= '/dbfs/CIFAR10/models/'
par_model2000_name = 'cifar_2000pictures.h5'
par_modelall_name = 'cifar_allpictures.h5' 

par_experiment_name = 'cifar10'

# In case cell gets status "cancelled" after execution, uninstall libraries, restart cluster and reinstall libraries

#1.  Log metrics of models

##### 1a. Authenticate to Azure ML workspace (interactive, using AAD and browser)

In [0]:
import sys
import requests
import time
import base64
import datetime
import azureml.core
import shutil
import os, json
from azureml.core import Workspace
from azureml.core.run import Run
from azureml.core.experiment import Experiment
from azureml.core.model import Model
import azureml.core
from azureml.core.authentication import ServicePrincipalAuthentication

ws = Workspace(workspace_name = workspace,
               subscription_id = subscription_id,
               resource_group = resource_grp)

ws.get_details()

/databricks/python/lib/python3.6/site-packages/requests/__init__.py:104: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (2.3.0)/charset_normalizer (2.0.7) doesn't match a supported version!
 RequestsDependencyWarning)
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (cryptography 2.6.1 (/databricks/python3/lib/python3.6/site-packages), Requirement.parse('cryptography>=3.3.1; extra == "crypto"'), {'PyJWT'}).
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code R9EKYVNHM to authenticate.

--------------------------------------------------------------------------- 
 AuthenticationException Traceback (most recent call last)
 <command-224773701457115> in <module> () 
 16 ws = Workspace(workspace_name = workspace,
 17 subscription_id = subscription_id , 
 ---> 18 resource_group = resource_grp)
 19 
 20 ws . get_details ( ) 

 /databricks/python/lib/python3.6/site-packages/azureml/core/workspace.py in __init__ (self, subscription_id, resource_group, workspace_name, auth, _location, _disable_service_check, _workspace_id, sku, tags, _cloud) 
 191 """
 192 if not auth : 
 --> 193 auth = InteractiveLoginAuthentication ( ) 
 194 
 195 self . _auth = auth 

 /databricks/python/lib/python3.6/site-packages/azureml/core/authentication.py in __init__ (self, force, tenant_id, cloud) 
 537 print("Performing interactive authentication. Please follow the instructions "
 538 "on the terminal.")
 --> 539 perform_interactive_login ( tenant = tenant_id , cloud_type = self . _cloud_type ) 
 540 print ( "Interactive authentication successfully completed." ) 
 541 

 /databricks/python/lib/python3.6/site-packages/azureml/_base_sdk_common/common.py in perform_interactive_login (username, password, service_principal, tenant, allow_no_subscriptions, identity, use_device_code, use_cert_sn_issuer, cloud_type) 
 587 if 'Server returned error in RSTR - ErrorCode' in msg : 
 588 raise AuthenticationException ( "Logging in through command line is not supported. " + suggestion ) 
 --> 589 raise AuthenticationException ( 'Unknown error occurred during authentication. Error detail: ' + str ( err ) ) 
 590 except requests . exceptions . ConnectionError as err : 
 591 raise AuthenticationException ( 'Please ensure you have network connection. Error detail: ' + str ( err ) ) 

 AuthenticationException : AuthenticationException:
	Message: Unknown error occurred during authentication. Error detail: Unexpected polling state code_expired
	InnerException None
	ErrorResponse 
{
 "error": {
 "code": "UserError",
 "inner_error": {
 "code": "Authentication"
 },
 "message": "Unknown error occurred during authentication. Error detail: Unexpected polling state code_expired"
 }
}

##### 1b. Load models from disk where it was stored in previous notebook 1_DeepLearningCifar!0NotebookExploration.py

In [0]:
import keras
from keras.models import load_model

#path= '/dbfs/CIFAR10/models/'
model2000path = path + par_model2000_name
modelallpath = path + par_modelall_name

model2000 = load_model(model2000path)
modelall = load_model(modelallpath)

In [0]:
model2000.summary()

##### 2c. Get testdate to regenerate metrics

In [0]:
from keras.datasets import cifar10
num_classes = 10
# The data, shuffled and split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
#2
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
#3
x_train /= 255
x_test /= 255

##### 2d. Create new experiment in Azure ML service workspace and add both models

In [0]:
# upload the serialized model into run history record
#mdl, ext = par_model_name.split(".")
#model_zip = mdl + ".zip"
#shutil.make_archive('/dbfs/'+ mdl, 'zip', path)
# start a training run by defining an experiment
myexperiment = Experiment(ws, par_experiment_name)
run = myexperiment.start_logging()

score2000 = model2000.evaluate(x_test, y_test, verbose=0)
scoreall = modelall.evaluate(x_test, y_test, verbose=0)

run.log_list("Test accuracy 2000 pics, all pics", [score2000[1], scoreall[1]])
run.log_list("Test loss 2000 pics, all pics", [score2000[0], scoreall[0]])

run.upload_file("outputs/" + par_model2000_name, model2000path)
run.upload_file("outputs/" + par_modelall_name, modelallpath)

run.complete()
run_id = run.id
print ("run id:", run_id)

#2. Register best model (trained with 60000 pictures)

In [0]:
registermodelall = Model.register(
    model_path=modelallpath,  # this points to a local file
    model_name=par_modelall_name,  # this is the name the model is registered as
    tags={"area": "spark", "type": "deeplearning", "run_id": run_id},
    description="Keras deeplearning, all pictures",
    workspace=ws,
)
print("Model registered: {} \nModel Description: {} \nModel Version: {}".format(registermodelall.name, registermodelall.description, registermodelall.version))